In [1]:
import json
import getpass
import requests
import wget
import sys
from usgs import api
import pandas as pd

# get api key

In [94]:
#Form JSON Object to store credentials to generate API Key.
username='rbavery'
password = getpass.getpass()

login = r'https://earthexplorer.usgs.gov/inventory/json/v/1.4.0/login'

creds={"username": username,
"password": password,
"authType": "",
"catalogId": "EE"}

json_data = {'jsonRequest' : json.dumps(creds)}

r = requests.post(login, data = json_data)
token = r.json()['data'].replace("'",'"') # json needs double quotes

if token == None:
    print("API Key Failed to Generate")

else:
    print("Your API Key Generated Successfully")
    print(token)
    
def check_item_basket(token):
    item_basket_json = {
        "apiKey" : token
    }


    get_bd =api_string+"itembasket"

    json_data = {'jsonRequest' : json.dumps(item_basket_json)}

    r = requests.post(get_bd, data = json_data)
    return r.json()

def submit_bulk_order(token):
    request_json = {
        "apiKey" : token
    }


    get_bd =api_string+"submitbulkdownloadorder"

    json_data = {'jsonRequest' : json.dumps(request_json)}

    r = requests.post(get_bd, data = json_data)
    return r.json()

#If Key fails to generate please see documentmentation to decode error https://earthexplorer.usgs.gov/inventory/documentation/errors

 ·········


Your API Key Generated Successfully
1c363ec39f684a54ac775c0617a8bee8


# make a geojson feature

In [3]:
geojson_aoi = feature_coll = {
"type": "FeatureCollection",
"features": [
{
"type": "Feature",
"properties": {},
"geometry": {
"type": "Polygon",
"coordinates": [
[
[
-104.1064453125,
43.13306116240612
],
[
-104.12841796875,
41.04621681452063
],
[
-102.19482421875,
41.0130657870063
],
[
-102.15087890624999,
40.027614437486655
],
[
-95.07568359375,
40.01078714046552
],
[
-95.625,
40.730608477796636
],
[
-96.13037109375,
42.17968819665961
],
[
-97.27294921875,
43.02071359427862],
[
-98.02001953125,
42.90816007196054
],
[
-98.54736328125,
43.08493742707592
],
[
-104.1064453125,
43.13306116240612
]
]
]
}
}
]
}

# get dataset fields

In [18]:
def make_date_strings(start_date, end_date):

    return [i.date() for i in pd.date_range(start_date, end_date).tolist() if i.month in [6, 7, 8, 9]]

The wrapper of the JSON API can be pip installed from my fork, required for geojson search

`!pip install git+https://github.com/rbavery/usgs`

In [27]:
import numpy as np
start_dates = [str(i) + "-06-01" for i in np.arange(2002, 2020)]
end_dates = [str(i) + "-09-30" for i in np.arange(2002, 2020)]

In [31]:
all_results = []
for start_date, end_date in zip(start_dates, end_dates):
    results = api.search(dataset="ARD_TILE", node="EE", start_date=start_date, end_date=end_date, ll={"longitude":-104.1064453125, "latitude":39.825413103424786}, ur={"longitude":-95.130615234375, "latitude":43.100982876188546}, api_key=token)
    all_results.extend(results['data']['results'])
    print(start_date+" done")

2002-06-01 done
2003-06-01 done
2004-06-01 done
2005-06-01 done
2006-06-01 done
2007-06-01 done
2008-06-01 done
2009-06-01 done
2010-06-01 done
2011-06-01 done
2012-06-01 done
2013-06-01 done
2014-06-01 done
2015-06-01 done
2016-06-01 done
2017-06-01 done
2018-06-01 done
2019-06-01 done


In [63]:
filtered_results = []
for i in all_results:
    if i['cloudCover'] < 10:
        if "landsat_etm_c1" in i['browseUrl']:
            pass
        else:
            filtered_results.append(i)

In [82]:
entity_ids = []
for i in filtered_results:
    entity_ids.append(i['entityId'])

In [128]:
download_opt_json = {
    "datasetName" : "ARD_TILE",
    "apiKey" : token,
    "entityIds": [entity_ids[0]]
}

api_string = r'https://earthexplorer.usgs.gov/inventory/json/v/1.4.0/'
get_bd =api_string+"downloadoptions"

json_data = {'jsonRequest' : json.dumps(download_opt_json)}

r = requests.post(get_bd, data = json_data)

In [148]:
bulk_order_request = {
    "datasetName": "ARD_TILE",
    "apiKey": token,
    "entityIds": entity_ids
}


get_bd =api_string+"getbulkdownloadproducts"

json_data = {'jsonRequest' : json.dumps(bulk_order_request)}

r = requests.post(get_bd, data = json_data)

bd_products = r.json()

In [132]:
download_json = {
    "datasetName" : "ARD_TILE",
    "products" : ["Surface Reflectance",
                  "Provisional Surface Temperature",
                  "Brightness Temperature",
                  "Quality Assessment",
                  "Full Resolution Browse",
                  "Metadata"],
    "apiKey" : token,
    "entityIds": entity_ids
}


In [133]:
# doesn't work, ard needs to be bulk ordered
get_bd =api_string+"download"

json_data = {'jsonRequest' : json.dumps(download_json)}

r = requests.post(get_bd, data = json_data)

In [137]:
r.json()

{'errorCode': None,
 'error': '',
 'data': [],
 'api_version': '1.4.0',
 'access_level': 'approved',
 'catalog_id': 'EE',
 'executionTime': 245.79620599746704}

In [159]:
lst_of_update_requests = [ {
    "apiKey": token,
    "datasetName": "ARD_TILE",
    "entityId": product_set["entityId"],
    "downloadCodes": [i['downloadCode'] for i in product_set["products"]]
} for product_set in bd_products['data'] ]

In [ ]:
# regular download doesn't work, ard needs to be bulk ordered
get_bd =api_string+"updatebulkdownloadscene"

for request in lst_of_update_requests:
    json_data = {'jsonRequest' : json.dumps(request)}

    r = requests.post(get_bd, data = json_data)

In [ ]:
api.dataset_fields("ARD_TILE", node="EE", api_key=token)

# There should be more results returned from Nebraska (the geojson used to search) throughout the year, I've confirmed this by downloading ARD manually.

# scratch code below

In [ ]:
all_results_good_tiles = []
for i in all_results:
    horizontal_grid = i['browseUrl'].split('/')[8]
    vertical_grid = i['browseUrl'].split('/')[9]
    if horizontal_grid in ["011","012","013","014","015","016","017"] and vertical_grid in ["006","007","008","009","010"]:
        all_results_good_tiles.append(i)

In [ ]:
where_and_dict = {
     21787 : ["011","012","013","014","015","016","017"], # tile grid horizontal
     21788 : ["006","007","008","009", "010"], # tile grid vertical
     21791 : '10', # cloud cover
     21792 : ['OLI_TIRS', 'TM']
}

additionalCriteria = {
    "filterType": "and",
    "childFilters": [
        {
            "filterType": "value",
            "fieldId": field_id,
            "value": value,
            "operand": "="
        }
        for field_id, value in iter(where.items())
    ]
}

In [ ]:
searchUrl='https://earthexplorer.usgs.gov/inventory/json/v/1.4.1/search'
dataset_search = {"apiKey": (token), "datasetName": "ARD_TILE"}
dataset_request =requests.get(searchUrl, params={"jsonRequest":json.dumps(dataset_search)})
d_resultslist = dataset_request.json()['data']['results']
order = []
for res in d_resultsList:
    order.append(res['entityId'])

#Check to see if query resulted in scenes being returned.
if len(order) == 0:
    sys.exit("No Scenes Matching Criteria Were Found, Program will terminate")
else:
    print("Your Scene List Contains:")
    print(order)

In [22]:
import pandas as pd

In [43]:
def read_dataset_json(fname):
    return pd.DataFrame(pd.read_json(fname).data.apply(lambda x: x['datasetName']))

In [4]:
#Create routine to generate download URLs for desired product for enity ids from search query. Please consult Get Bulk Download
#product endpoint for available products per dataset. Here Level-1 Landsat Geotiff will be downloaded. The product is downloaded
#using wget.

for enityID in [order]:
  orderParms ={'datasetName':dataset,
          'products':'STANDARD',
          'entityIds': order,
          "apiKey": (token)
          
          }
  
  downloadURL="https://earthexplorer.usgs.gov/inventory/json/v/1.4.0/download"
  download= requests.get(downloadURL, params={'jsonRequest':json.dumps(orderParms)})
    
urlList = []
download_list = download.json()['data']
for item in download_list:
    urlList.append(item['url'])

#Users will need to specify a download directory on their machine.
outDir="YOUR DIR"
for URL in urlList:
    wget.download(URL, out=outDir, bar=None)
print("The Following Scenes have downloaded:")
print(order)
print("Download Complete")

The Following Scenes have downloaded:
['LC80260272014047LGN01', 'LC80260272014063LGN01', 'LC80260272014159LGN01', 'LC80260272014207LGN01', 'LC80260272014319LGN01']
Download Complete
